In [ ]:
# next todos:
# construct a vector for one bounding box
#    - fetch all the features within a bb (allready works) [x]
#    - wheig the features according to their median size in sample dataset
#    - feed words acording to weiths into a document 
#    - generate a vector for this document

In [3]:
import parse_osm
import tag_handler
from shapely.geometry import Polygon, LineString, Point, MultiPolygon, MultiLineString, MultiPoint
from shapely.ops import transform
import random
import pandas as pd
import geopandas as gpd
import pyproj
import numpy as np
import statistics

In [104]:
tag_df =  pd.read_pickle('../data/spatial_semantics/kv_df_just_eng.pickle')
tag_median_size_df = pd.read_pickle('../data/spatial_semantics/tag_sizes_median_df.pickle').set_index(['key','value']).sort_index()
#replace pd nan with None
tag_median_size_df = tag_median_size_df.where(pd.notnull(tag_median_size_df), None)
filtertags_handler = tag_handler.filtertags(tag_df)#TU Wien polygon
# from here on out we want key value in the tag_df as the index because it makes
# the look up faster
# we sort the index to handle the "1: PerformanceWarning: indexing past lexsort depth may impact performance." Warning
tag_df = tag_df.set_index(['key','value']).sort_index()
q_poly = Polygon([(16.36896371841431,48.20063653233946),(16.373598575592045,48.19960677385028),(16.371034383773807,48.19771882952509),(16.36772990226746,48.19816221664037)])

In [107]:
def compose_document(element):
    # example element:
    # {'leisure pitch': {'area': 373.27528143301606}, 'sport basketball': {'area': 373.27528143301606}}
    doc_shard = ''
    for key in element.keys():
        group, sub_type =  key.split(' ')
        # we lookup the combination in the tag_df
        row = tag_df.loc[group, sub_type]
        # and retrieve both the wikidata and the wikipedia text from it
        wikidata_desc = row.iloc[0].wikidata_desc
        wikipedia_desc = row.iloc[0].en_text
        
        # now we test if we aktualy have wikipedia text
        if wikipedia_desc:
            text = wikipedia_desc
        # if not we take the wikidata description instead
        else:
            text = wikidata_desc
        
        # now we test if we have an area or a lenght for the group sub_type combo
        if element[key]:
            # this is uqly and needs restructuring:
            if element[key].get('area'):
                area = element[key].get('area')
                # now we need to get the median area for this group sub_type combo
                try:
                    m_area = tag_median_size_df.loc[group,sub_type].median_area
                except:
                    m_area = None
                
                if m_area:
                    doc_shard += text_weigher(text,area/m_area)
                else:
                    # if we dont have an median we wiegh with 0
                    doc_shard += text_weigher(text,0)  
            
            if element[key].get('length'):
                length = element[key].get('length')
                # now we need to get the median area for this group sub_type combo
                try:
                    m_length = tag_median_size_df.loc[group,sub_type].median_length
                except:
                    m_length = None
                
                if m_length:
                    doc_shard += text_weigher(text,length/m_length)
                else:
                    # if we dont have an median we wiegh with 0
                    doc_shard += text_weigher(text,0)                
        
        else:
            # else is the case when the element is just a node thus it does not
            # have an area or length
            doc_shard += text_weigher(text,0)

    return doc_shard



def text_weigher(text,weight):
    # for the time beeing we just round the weight to the next intiger
    weight = round(weight)
    
    if weight <= 1 :
        # we want the text atleast once 
        w_text = text
    
    else:
        text += ' '
        w_text = text*weight
        
    return w_text
    

In [32]:
osm_handle = parse_osm.disect_osm(parse_osm.json_from_osm(q_poly))
osm_handle.feature_df

In [85]:
a = 'a'*5
a

'aaaaa'

In [37]:
selected_tags_df = osm_handle.feature_df[osm_handle.feature_df.apply(filtertags_handler.filter_them, axis = 1)]
type_id_list = selected_tags_df.index.tolist()
# for the selected features we calculate the geometry:
[osm_handle.get_geometry(f_type,osm_id) for f_type, osm_id in type_id_list]
# reselect to fetch the generated geometries
selected_tags_df = osm_handle.feature_df[osm_handle.feature_df.apply(filtertags_handler.filter_them, axis = 1)]
selected_tags_df

geometry_ref  \
type id                                                             
node 119403521                                               None   
     119403523                                               None   
     270929643                                               None   
     271334053                                               None   
     276372849                                               None   
...                                                           ...   
way  689093871  [1772629803, 6464060426, 6464060427, 646406042...   
     700415774                           [3993342688, 4135190909]   
     700415775                           [2028498703, 2028498790]   
     808894535  [60664062, 7563777116, 7563777117, 7563777119,...   
     848458964  [7917152352, 7917152350, 7917152349, 791715235...   

                                                         geometry  \
type id                                                             
node 119403521            {'point': POINT (16.369217 48.2002961)}   
     119403523           {'point': POINT (16.3696581 48.2003446)}   
     270929643           {'point': POINT (16.3689156 48.2002486)}   
     271334053           {'point': POINT (16.3695828 48.2004257)}   
     276372849           {'point': POINT (16.3716187 48.1990612)}   
...                                                           ...   
way  689093871  {'polygon': POLYGON ((16.3727613 48.1995461, 1...   
     700415774  {'line': LINESTRING (16.369697 48.2004685, 16....   
     700415775  {'line': LINESTRING (16.3696261 48.2003305, 16...   
     808894535  {'polygon': POLYGON ((16.3722629 48.1980862, 1...   
     848458964  {'polygon': POLYGON ((16.3678131 48.198383, 16...   

                                                             tags  
type id                                                            
node 119403521                              {'railway': 'switch'}  
     119403523  {'bus': 'yes', 'highway': 'bus_stop', 'local_r...  
     270929643  {'entrance': 'yes', 'entrance_marker:subway': ...  
     271334053  {'light_rail': 'yes', 'local_ref': 'H', 'name'...  
     276372849  {'artist:wikidata': 'Q151097', 'artist_name': ...  
...                                                           ...  
way  689093871        {'barrier': 'fence', 'leisure': 'dog_park'}  
     700415774  {'electrified': 'rail', 'electrified:rail': 's...  
     700415775  {'electrified': 'rail', 'electrified:rail': 's...  
     808894535                               {'landuse': 'grass'}  
     848458964  {'amenity': 'fast_food', 'building': 'kiosk', ...  

[325 rows x 3 columns]

In [111]:
combined_document = ''
for i,element in enumerate(selected_tags_df[['geometry','tags']].apply(filtertags_handler.calcualte_size_for_tags, axis=1).tolist()):
    # and then we add the document shard for a given element
    combined_document += compose_document(element)
    # some padding
    combined_document += ' '

In [116]:
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import multiprocessing
import pickle

In [117]:
doc2vec_model = Doc2Vec.load("../data/en_wikipedia_corpus/doc2vec_eng.pickel")

In [120]:
vec = doc2vec_model.infer_vector(combined_document.split(' '))

In [124]:
doc2vec_model.docvecs?

Type:            Doc2VecKeyedVectors
String form:     <gensim.models.keyedvectors.Doc2VecKeyedVectors object at 0x7fcf8292e5e0>
Length:          4965789
File:            ~/.local/lib/python3.8/site-packages/gensim/models/keyedvectors.py
Docstring:       <no docstring>
Class docstring: Abstract base class / interface for various types of word vectors.


In [31]:
import ipyleaflet
from ipyleaflet import Map, basemaps, basemap_to_tiles, GeoJSON, WKTLayer
from ipywidgets import Label




m = Map(
    basemap=basemaps.CartoDB.Positron,
    # for some reason lat lon are switch for centering the map
    center=((q_poly.centroid.coords[0][1],q_poly.centroid.coords[0][0])),
    #center=((test_multi['multipolygon'].centroid.coords[0][1],test_multi['multipolygon'].centroid.coords[0][0])),
    zoom=14
)

wlayer = WKTLayer(
    wkt_string=q_poly.wkt,
    #hover_style={"fillColor": "red"},
    fill_color="red",
    color="red",
)
m.add_layer(wlayer)


# m.add_layer(geo_json)


m

Map(center=[48.19911188084065, 16.370350493313698], controls=(ZoomControl(options=['position', 'zoom_in_text',…